https://dl.acm.org/doi/10.1145/3633211 Shoggoth: A Formal Foundation for Strategic Rewriting

https://spoofax.dev/background/stratego/strategic-rewriting/strategic-rewriting/
https://spoofax.dev/references/stratego/

https://github.com/sdiehl/pyrewrite

https://en.wikipedia.org/wiki/XSLT

https://inst.eecs.berkeley.edu/~cs294-260/sp24/2024-01-22-term-rewriting;./ max's notes
https://inst.eecs.berkeley.edu/~cs294-260/sp24/2024-01-31-opt-strategies elevate

https://arxiv.org/html/2411.06409v1 Automated Strategy Invention for Confluence of Term Rewrite Systems

env,ctx,t -> list (env, ctx, t)


t -> list t can be lifted

(ctx, t) -> list (ctx, t)

https://news.ycombinator.com/item?id=25112907

spoofax
elan

Eelco Visser, Zine-El-Abidine Benaissa, and Andrew P. Tolmach. Building program optimizers with rewriting strategies https://doi.org/10.1145/289423.289425

https://inria.hal.science/hal-01143486/document Rewriting Strategies and Strategic Rewrite Programs Kirchner

https://www.irif.fr/~letouzey/types2014/slides-13.pdf Proof-Relevant Rewriting Strategies - Matthieu Sozeau. Generalized rewriting strategies https://www.irif.fr/~letouzey/types2014/abstract-13.pdf



Minikanren vs minikanren-flp vs micro-stratego

unification -> matching, which is simpler.
State needs context (?)
We could reuse host ctx.

(subst, term) -> list (subst, term)

one thing is marked special. Could have special variable in subst to hold term. That's how you embed into minikanren.. ehh, not so much.

lambda-stratego
clp-stratego
asp-stratego
tabled-stratego
data-stratego

krishnaswami yallop style partial evaluation based on heads


type Ctx = QunCtx Ctx | DeclHole Ctx | Top 

https://github.com/JuliaSymbolics/Rewriters.jl

https://eelcovisser.org/publications/1998/VisserB98.pdf a core language for rewriting. System S

Progy graph strategy language

conversions https://www.cl.cam.ac.uk/~lp15/papers/Reports/TR035-lcp-rewriting.pdf

inferring strategies as proof objects? (compactify / generalize a concrete rewriting path)


Have zipper apply congruences
Conversions. Start from refl t = t and go left once. Zipper object is in charge of congruence.

Modulo Context.
Hash consing context + zipper. Syntax trees are not dags.
Hash conses perfectly destroy context.
(ctx, t)

https://www.scitepress.org/Papers/2024/127040/127040.pdf pyzstrageic 




Contextual rewriting using zipper.
forall lhs = rhs, pos  . Use pos as "trigger" location for indexing. Coindcutives, sets, etc
Match on context.

Algebraic effects system?
raise(Foo, )
catch()

f(raise(cont)) = raise(cont f)
Rules for evertyhintg. Hmm. Kind of reminds me of generalized rewriting?
f(raise) ~ effect congruence
Hmm.

type Effect = Read() | Write | Fork ()

With zipper, these things become more doable.




In [ ]:
type Subst = dict[smt.ExprRef, smt.ExprRef]
type Ctx = list[smt.ExprRef]
type State = tuple[Subst, Ctx, smt.ExprRer]
type Strategy = Generator[[State], State]
def pmatch(pat) -> Strategy:
    def res(state):

    return res

def build(schema) -> Stategy:
    def res(_t):
        yield smt.substitute(schema, *subst.items())
    return res
def seq(*strats): # conj
    def res(st):
        if len(strats) == 0:
            yield st
        else:
            for state in strats:
                yield from seq(*strats[1:])(state)
    return res

def fairseq(*strats)

def fail(state): return

def repeat(strat):

def all_child(strat): # mapchild?
def some(strat):

def topdown(): ...
def bottomup(): ...
def innermost(): ...

def id_(strat): return strat




def try_(strat):

def left(a, b):
    def res(state):
        n = 0
        for st in a(state):
            n += 1
            yield st
        if n == 0:
            yield from b(state)

class Strat():
    def __matmul__(self, rhs): ...
    def __add__(self, rhs):


In [ ]:
n,m = smt.Ints("n m")
fib = smt.Function("fib", smt.IntSort(), smt.IntSort())
fib = kd.define("fib", [n], smt.If(n == 0, 0, smt.If(n == 1, 1, fib(n-1) + fib(n-2))))





4181

In [ ]:
fib0 = kd.prove(fib(0) == 0, unfold=1)
fib1 = kd.prove(fib(1) == 1, unfold=1)
fib2 = kd.prove(fib(2) == 1, unfold=2)



In [ ]:
import kdrag.theories.nat as nat
nat.add_S
nat.add_Z

kd.rewrite.full_simp(nat.from_int(10))
#kd.rewrite.full_simp(nat.add(m, nat.S(nat.S(nat.S(nat.Z())))))
kd.rewrite.full_simp(nat.from_int(10) + )


S(S(S(S(S(S(S(S(S(S(Z))))))))))

In [ ]:
hundred = kd.rewrite.full_simp(nat.from_int(100))

In [67]:
%%timeit
kd.rewrite.full_simp(hundred + nat.Z)

168 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
hundred = kd.rewrite.full_simp(nat.from_int(100))

In [66]:
%%timeit
kd.rewrite.rewrite(hundred + nat.Z, [nat.add_S, nat.add_Z])

319 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
kd.rewrite.full_simp(fib(19))

7.03 ms ± 59 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
@dataclass(frozen=True)
class QuantifierHole():
    vs : list[smt.ExprRef] # orig_vs : list[smt.ExprRef]
class LambdaHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.Lambda(self.vs, e)
class ForAllHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.ForAll(self.vs, e)
class ExistsHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.Exists(self.vs, e)

@dataclass(frozen=True)
class DeclHole():
    f : smt.FuncDeclRef
    _left : tuple[smt.ExprRef]
    _right : tuple[smt.ExprRef]
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return self.f(*self._left, e, *self._right)
    def left(self, t : smt.ExprRef) -> tuple[Ctx, smt.ExprRef]:
        assert len(self._left) > 0
        return DeclHole(self.f, self._left[:-1], (self._left[-1],) + self._right), self._left[-1]
    def right(self, t : smt.ExprRef) -> tuple[DeclHole, smt.ExprRef]:
        assert len(self._right) > 0
        return DeclHole(self.f, self._left + (self._right[0],), self._right[1:]), self._right[0]
    def has_left(self) -> bool:
        return len(self._left) > 0
    def has_right(self) -> bool:
        return len(self._right) > 0
type Hole = ForAllHole | ExistsHole | LambdaHole | DeclHole

@dataclass
class Zipper():
    ctx : list[Hole] # trail / stack
    t : smt.ExprRef
    @classmethod
    def from_term(cls, t : smt.ExprRef) -> "Zipper":
        return cls([], t)
    def pop(self): # up?
        hole = self.ctx.pop()
        self.t = hole.wrap(self.t)
        return self
    def copy(self):
        return Zipper(self.ctx.copy(), self.t)
    def left(self):
        hole = self.ctx.pop()
        assert isinstance(hole, DeclHole)
        hole, self.t = hole.left(self.t)
        self.ctx.append(hole)
        return self
    def right(self):
        assert isinstance(self.ctx, DeclHole)
        self.ctx, self.t = self.ctx.right(self.t)
        return self
    def arg(self, n):
        children = self.t.children()
        self.ctx.append(DeclHole(self.t.decl(), children[:n],  children[n+1:]))
        self.t = children[n]
        return self
    def open_binder(self):
        assert isinstance(self.t, smt.QuantifierRef)
        vs, body = kd.utils.open_binder(self.t)
        if self.t.is_forall():
            hole = ForAllHole(vs)
        elif self.t.is_exists():
            hole = ExistsHole(vs)
        elif self.t.is_lambda():
            hole = LambdaHole(vs)
        self.ctx.append(hole)
        self.t = body
        return self
    def __hash__(self):
        return hash((tuple(self.ctx), self.t))


x,y,z = smt.Ints('x y z')
Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20)))).open_binder().arg(0).arg(1).pop().pop().pop().t


{Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20)))) : 3}
Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20)))) == Zipper.from_term(smt.ForAll([x], smt.Implies(y > 0, smt.And(x < 10, x < 20))))
{Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20)))) : 3}[Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20))))]


3

In [ ]:
from kdrag.all import *
from dataclasses import dataclass
from typing import Protocol
class Ctx(Protocol):
    ctx : Optional["Ctx"]
    def pop(self, e:smt.ExprRef) -> smt.ExprRef: ...
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef: ...
    def arg(self, n:int, t : smt.ExprRef) -> tuple[Ctx, smt.ExprRef]:
        assert smt.is_app(t)
        children = t.children()
        f = t.decl()
        return DeclHole(f, children[:n], children[n+1:], self), children[n]
    def children(self) -> list[tuple[Ctx, smt.ExprRef]]:
        return [self.arg(n) for n in range(len(self.body.children()))]
    def open_binder(self, t : smt.QuantifierRef) -> tuple[Ctx, smt.ExprRef]:
        assert isinstance(t, smt.QuantifierRef)
        vs, body = kd.utils.open_binder(t)
        if t.is_forall():
            return ForAllHole(vs, body, self), body
        elif t.is_exists():
            return ExistsHole(vs, body, self), body
        elif t.is_lambda():
            return LambdaHole(vs, body, self), body
        else:
            raise NotImplementedError(f"Unknown binder {t}")

@dataclass(frozen=True)
class Hole(Ctx):
    ctx = None
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        raise NotImplementedError()   
@dataclass(frozen=True)
class QuantifierHole(Ctx):
    vs : list[smt.ExprRef] # orig_vs : list[smt.ExprRef]
    ctx : Ctx
class LambdaHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.Lambda(self.vs, e)
class ForAllHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.ForAll(self.vs, e)
class ExistsHole(QuantifierHole):
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return smt.Exists(self.vs, e)

@dataclass(frozen=True)
class DeclHole(Ctx):
    f : smt.FuncDeclRef
    _left : tuple[smt.ExprRef]
    _right : tuple[smt.ExprRef]
    ctx : Ctx
    def wrap(self, e:smt.ExprRef) -> smt.ExprRef:
        return self.f(*self._left, e, *self._right)
    def left(self, t : smt.ExprRef) -> tuple[Ctx, smt.ExprRef]:
        assert len(self._left) > 0
        return DeclHole(self.f, self._left[:-1], (self._left[-1],) + self._right, self.ctx), self._left[-1]
    def has_left(self) -> bool:
        return len(self._left) > 0
    def has_right(self) -> bool:
        return len(self._right) > 0
    def right(self, t : smt.ExprRef) -> tuple[Ctx, smt.ExprRef]:
        assert len(self._right) > 0
        return DeclHole(self.f, self._left + (self._right[0],), self._right[1:], self.ctx), self._right[0]

@dataclass
class Zipper():
    ctx : Ctx
    t : smt.ExprRef
    @classmethod
    def from_term(cls, t : smt.ExprRef) -> "Zipper":
        return cls(Hole(), t)
    def pop(self): # up?
        self.ctx, self.t = self.ctx.wrap(self.t)
        return self
    def left(self):
        assert isinstance(self.ctx, DeclHole)
        self.ctx, self.t = self.ctx.left(self.t)
        return self
    def right(self):
        assert isinstance(self.ctx, DeclHole)
        self.ctx, self.t = self.ctx.right(self.t)
        return self
    def arg(self, n):
        self.ctx, self.t = self.ctx.arg(n, self.t)
        return self
    def open_binder(self):
        self.ctx, self.t = self.ctx.open_binder(self.t)
        return self


x,y,z = smt.Ints('x y z')

Zipper.from_term(smt.ForAll([x], smt.Implies(x > 0, smt.And(x < 10, x < 20)))).open_binder().arg(0).arg(1).pop()





NotImplementedError: No implementation of getitem for sort Bool. Register a definition via getitem.register(Bool, your_code)